In [ ]:
%pip install tensorflow
%pip install PIL

In [ ]:
import tensorflow as tf
import keras as K
import warnings
import numpy as np
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image as PilImage

warnings.filterwarnings("ignore")

In [ ]:
# Inicializando a Rede Neural Convolucional
classifier = Sequential()

# Passo 1 - Primeira Camada de Convolução
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Passo 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adicionando a Segunda Camada de Convolução
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Passo 3 - Flattening
classifier.add(Flatten())

# Passo 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compilando a rede
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


Criando os objetos train_datagen e validation_datagen com as regras de pré-processamento das imagens


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

validation_datagen = ImageDataGenerator(rescale = 1./255)

Pré-processamento das imagens de treino e validação

In [ ]:
training_set = train_datagen.flow_from_directory('training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

validation_set = validation_datagen.flow_from_directory('test_set',
                                                        target_size = (64, 64),
                                                        batch_size = 32,
                                                        class_mode = 'binary')

In [ ]:
def custom_generator(generator):
    while True:
        data = next(generator)
        yield data

repeated_train = custom_generator(training_set)

Executando o treinamento (esse processo pode levar bastante tempo, dependendo do seu computador)

In [ ]:
classifier.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
                   loss='binary_crossentropy', metrics=['accuracy'])

classifier.fit(repeated_train,
                         steps_per_epoch = 100,
                         epochs = 5,
                         validation_data = validation_set,
                         validation_steps = 25)

Primeira Imagem

In [ ]:
path = 'test_set/bart/bart17.bmp'
img = PilImage.open(path)
img.save(path.replace('.bmp', '.png'))

test_image = image.load_img(path.replace('.bmp', '.png'), target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices

if result[0][0] == 1:
    prediction = 'Homer'
else:
    prediction = 'Bart'

prediction


Segunda Imagem

In [ ]:
path = 'test_set/homer/homer17.bmp'
img = PilImage.open(path)
img.save(path.replace('.bmp', '.png'))

test_image = image.load_img(path.replace('.bmp', '.png'), target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices

if result[0][0] == 1:
    prediction = 'Homer'
else:
    prediction = 'Bart'

prediction

Terceira Imagem

In [ ]:
path = 'test_set/bart/bart30.bmp'
img = PilImage.open(path)
img.save(path.replace('.bmp', '.png'))

test_image = image.load_img(path.replace('.bmp', '.png'), target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices

if result[0][0] == 1:
    prediction = 'Homer'
else:
    prediction = 'Bart'

prediction

Avaliar o modelo

In [ ]:
score = classifier.evaluate(validation_set, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])